In [2]:
# %load bigquery_27.py
import six
from google.cloud import bigquery

client = bigquery.Client()

In [29]:
query = """
 SELECT
    C.id as id, repo_name, path, content
  FROM (
  SELECT id, content FROM
    [bigquery-public-data:github_repos.contents]
  WHERE
    content like '%k8s%' or content like '%docker' or content like '%kubernetes%'
    GROUP BY ID, CONTENT
  ) AS C
INNER JOIN (
  SELECT
    id, repo_name, path
  FROM
    [bigquery-public-data:github_repos.files]
  WHERE
    LOWER(path) LIKE '%{}' and LOWER(path) not LIKE '%vendor%'
  GROUP BY
    path, id, repo_name) AS F
ON
      C.id = F.id
""".format('gopkg.toml')

In [30]:
def run_query(query):
    query_results = client.run_sync_query(query)
    query_results.use_legacy_sql = True

    query_results.run()

    # The query might not complete in a single request. To account for a
    # long-running query, force the query results to reload until the query
    # is complete.
    while not query_results.complete:
      query_iterator = query_results.fetch_data()
      try:
         six.next(iter(query_iterator))
      except StopIteration:
          pass

    rows = query_results.fetch_data()
    return rows

In [31]:
response = run_query(query)

In [32]:
resp = []
for row in response:
    resp.append(row)

In [33]:
with open('kube_ecosystem_gopkg_toml.json', 'w') as f:
    f.write(json.dumps(resp))

In [34]:
print(len(resp))

265


In [35]:
file_list = ['kube_ecosystem_glide_yaml.json', 'kube_ecosystem_godeps.json', 'kube_ecosystem_gopkg_toml.json']

In [38]:
for fname in file_list:
    with open(fname) as f:
        jc = json.load(f)
    print(fname, "|", len(jc))

kube_ecosystem_glide_yaml.json | 564
kube_ecosystem_godeps.json | 6894
kube_ecosystem_gopkg_toml.json | 265


In [39]:
for fname in file_list:
    schema_rep = []
    with open(fname) as f:
        file_content = json.load(f)
        for tuple_ in file_content:
            schema_rep.append({
                "id": tuple_[0],
                "repo_name": tuple_[1],
                "path": tuple_[2],
                "content": tuple_[3]
            })
        with open("schema" + fname, 'w') as o:
            o.write(json.dumps(schema_rep))